# **RVC-CLI para treino (Versão simplificada)**
Criado por [ShiromiyaG](https://github.com/ShiromiyaG)
- Usa o [Applio](https://github.com/IAHispano/Applio) do [IAHispano](https://github.com/IAHispano)

In [ ]:
#@title # **Instalar**

from google.colab import drive

drive.mount("/content/drive")

!git clone https://github.com/IAHispano/Applio.git --branch 3.2.2 --single-branch
%cd /content/Applio
rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)
import codecs
import os
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime
E = Exception
B = print


def vidal_setup(ForceIn):
    L = "Kikpm.ovm.bu"
    K = "/content/"
    C = ForceIn

    def F():
        print("Installing pip packages...")
        subprocess.check_call(["pip", "install", "-r", "requirements.txt", "--quiet"])

    A = K + rot_47(L)
    G = K + rot_47(L)
    D = "/"
    if not os.path.exists(A):
        M = os.path.dirname(A)
        os.makedirs(M, exist_ok=True)
        print("No cached install found..")
        try:
            N = rot_47(
                codecs.decode(
                    "pbbxa://pcooqvonikm.kw/QIPqaxivw/Ixxtqw/zmawtdm/uiqv/Kwtij/Xvxcz.biz.oh",
                    "rot_13",
                )
            )
            subprocess.run(["wget", "-O", A, N])
            print("Download completed successfully!")
        except E as H:
            print(str(H))
            if os.path.exists(A):
                os.remove(A)
    if Path(A).exists():
        with tarfile.open(G, "r:gz") as I:
            for J in I.getmembers():
                O = os.path.join(D, J.name)
                try:
                    I.extract(J, D)
                except E as H:
                    print("Failed to extract a file")
                    C = True
            print(f"Extraction of {G} to {D} completed.")
        if os.path.exists(A):
            os.remove(A)
        if C:
            F()
            C = False
    else:
        F()


vidal_setup(False)
!python core.py prerequisites --pretraineds_v1 "False" --pretraineds_v2 "True" --models "True" --exe "False"
print("Finished installing requirements!")

# **Treino**

In [ ]:
#@title ### Preprocessar dataset e Extract Features
# @markdown ### ➡️ Informações do modelo
nome_do_modelo = "" # @param {type:"string"}
#@markdown ##### **Caminho da pasta no seu drive**
caminho_para_o_dataset = "rvc-cli/nome-do-dataset"  # @param {type:"string"}
caminho_completo = "/content/drive/MyDrive/" + caminho_para_o_dataset
sample_rate = 32000 # @param [32000, 40000, 48000] {allow-input: false}
cpu_cores = 2
!python core.py preprocess --model_name "{nome_do_modelo}" --dataset_path "{caminho_completo}" --sample_rate "{sample_rate}" --cpu_cores "{cpu_cores}"
versao_do_rvc = "v2"  # @param ["v2", "v1"] {allow-input: false}
metodo_do_f0 = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}
orientação_de_pitch = True  # @param{type:"boolean"}
tamanho_do_hop = 128  # @param {type:"slider", min:1, max:512, step:0}

!python core.py extract --model_name "{nome_do_modelo}" --rvc_version "{versao_do_rvc}" --f0_method "{metodo_do_f0}" --pitch_guidance "{orientação_de_pitch}" --hop_length "{tamanho_do_hop}" --sample_rate "{sample_rate}" --cpu_cores "{cpu_cores}" --gpu "0"

In [ ]:
# @title ### Treino
# @markdown ### ➡️ Informações do modelo
import threading
import time
import os
import shutil
import hashlib
import time

LOGS_FOLDER = "/content/Applio/logs/"
WEIGHTS_FOLDER = LOGS_FOLDER + nome_do_modelo
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVC_Backup"


def import_google_drive_backup():
    print("Importando dataset do Google Drive...")
    weights_exist = False
    for root, dirs, files in os.walk(GOOGLE_DRIVE_PATH):
        for filename in files:
            filepath = os.path.join(root, filename)
            if os.path.isfile(filepath) and not filepath.startswith(
                os.path.join(GOOGLE_DRIVE_PATH, "weights")
            ):
                backup_filepath = os.path.join(
                    LOGS_FOLDER, os.path.relpath(filepath, GOOGLE_DRIVE_PATH)
                )
                backup_folderpath = os.path.dirname(backup_filepath)
                if not os.path.exists(backup_folderpath):
                    os.makedirs(backup_folderpath)
                    print(f"Criar pasta de backup: {backup_folderpath}", flush=True)
                shutil.copy2(filepath, backup_filepath)  # copy file with metadata
                print(f"Importar arquivo de backup do Google Drive: {filename}")
            elif filepath.startswith(
                os.path.join(GOOGLE_DRIVE_PATH, "weights")
            ) and filename.endswith(".pth"):
                weights_exist = True
                weights_filepath = os.path.join(
                    WEIGHTS_FOLDER,
                    os.path.relpath(
                        filepath, os.path.join(GOOGLE_DRIVE_PATH, "weights")
                    ),
                )
                weights_folderpath = os.path.dirname(weights_filepath)
                if not os.path.exists(weights_folderpath):
                    os.makedirs(weights_folderpath)
                    print(f"Criar pasta dos weights: {weights_folderpath}", flush=True)
                shutil.copy2(filepath, weights_filepath)  # copy file with metadata
                print(f"Importar arquivo de weights: {filename}")
    if weights_exist:
        print("Copiado os weights do Google Drive para pasta de weights local.")
    else:
        print("Nenhum weights encontrado no backup do Google Drive.")
    print("Importação do backup do Google Drive concluída.")


def get_md5_hash(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def copy_weights_folder_to_drive():
    destination_folder = os.path.join(GOOGLE_DRIVE_PATH, "weights")
    try:
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        num_copied = 0
        for filename in os.listdir(WEIGHTS_FOLDER):
            if filename.endswith(".pth"):
                source_file = os.path.join(WEIGHTS_FOLDER, filename)
                destination_file = os.path.join(destination_folder, filename)
                if not os.path.exists(destination_file):
                    shutil.copy2(source_file, destination_file)
                    num_copied += 1
                    print(f"{filename} copiado para o Google Drive!")

        if num_copied == 0:
            print("Não foram encontrados novos modelos acabados para copiar.")
        else:
            print(f"Terminou a cópia de {num_copied} arquivos para o Google Drive!")

    except Exception as error:
        print(f"Ocorreu um erro durante a cópia de pesos para o Google Drive: {str(error)}")


if "autobackups" not in globals():
    autobackups = False


def backup_files():
    print("\nComeçando loop de backups...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated = False
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" directory
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")
                print("Criando backup de arquivos, espere")
                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                                print(
                                    f"Criado pasta de backups: {backup_folderpath}",
                                    flush=True,
                                )
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(
                                    current_timestamp
                                )
                                if last_backup_timestamp is None:
                                    #print(f"Criado backup do arquivo: {filename}")
                                    pass
                                else:
                                    print(f"Atualizando backup do arquivo: {filename}")
                                updated = True
                                fully_updated = False

            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        print(f"Arquivo deletado: {filepath}")
                    del last_backup_timestamps[filepath]
                    updated = True
                    fully_updated = False

            if not updated and not fully_updated:
                print("Arquivos estão atualizados.")
                fully_updated = True
                sleep_time = 15
            else:
                sleep_time = 0.1

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(sleep_time)

        except Exception as error:
            print(f"Ocorreu um erro durante backup: {str(error)}")


if autobackups:
    autobackups = False
    print("Autobackup desativado")
else:
    autobackups = True
    print("Autobackup ativado")

total_de_epoch = 250  # @param {type:"integer"}
batch_size = 16  # @param {type:"slider", min:1, max:25, step:0}
gpu = 0
orientação_de_pitch = True  # @param{type:"boolean"}
auto_backups = True  # @param{type:"boolean"}
pretrained = True
sincronizar_graficos = False  # @param{type:"boolean"}
dados_em_cache_na_gpu = False  # @param{type:"boolean"}
tensorboard = True  # @param{type:"boolean"}
# @markdown ### ➡️ Escolher de quantos em quantos Epochs seu modelo vai ser salvo
salvar_a_cada_epoch = 10  # @param {type:"slider", min:1, max:100, step:0}
salvar_somente_ultimo_G_e_D = True  # @param{type:"boolean"}
salvar_cada_weights = True  # @param{type:"boolean"}
detector_de_overtraining = False  # @param{type:"boolean"}
limiar_do_overtraining = 50  # @param {type:"slider", min:1, max:100, step:0}
# @markdown ### ➡️ Opcional
# @markdown Se você quiser um pretreino customizado
pretreino_customizado = False  # @param{type:"boolean"}
pretreino_escolhido = "Titan" # @param ["Titan", "Ov2", "Nanashi v1", "Nanashi v1.7"] {allow-input: false}
#@markdown ##### Os pretreino só estão disponiveis nos seguintes sample rates:
#@markdown ##### **Titan:** 32k, 40k, 48k
#@markdown ##### **Ov2:** 32k, 40k
#@markdown ##### **Nanashi v1:** 32k
#@markdown ##### **Nanashi v1.7:** 32k
def baixar_modelos(pretreino_escolhido, sample_rate):
    if pretreino_escolhido == "Titan":
        if sample_rate == 32000:
            !wget https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/32k/pretrained/G-f032k-TITAN-Medium.pth
            !wget https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/32k/pretrained/D-f032k-TITAN-Medium.pth
            g_pretrained_path = "/content/Applio/G-f032k-TITAN-Medium.pth"
            d_pretrained_path = "/content/Applio/D-f032k-TITAN-Medium.pth"
        elif sample_rate == 40000:
            !wget https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/40k/pretrained/G-f040k-TITAN-Medium.pth
            !wget https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/40k/pretrained/D-f040k-TITAN-Medium.pth
            g_pretrained_path = "/content/Applio/G-f040k-TITAN-Medium.pth"
            d_pretrained_path = "/content/Applio/D-f040k-TITAN-Medium.pth"
        elif sample_rate == 48000:
            !wget https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/48k/pretrained/G-f048k-TITAN-Medium.pth
            !wget https://huggingface.co/blaise-tk/TITAN/resolve/main/models/medium/48k/pretrained/D-f048k-TITAN-Medium.pth
            g_pretrained_path = "/content/Applio/G-f048k-TITAN-Medium.pth"
            d_pretrained_path = "/content/Applio/D-f048k-TITAN-Medium.pth"
    elif pretreino_escolhido == "Ov2":
        if sample_rate == 32000:
            !wget https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super32kG.pth
            !wget https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super32kD.pth
            g_pretrained_path = "/content/Applio/f0Ov2Super32kG.pth"
            d_pretrained_path = "/content/Applio/f0Ov2Super32kD.pth"
        elif sample_rate == 40000:
            !wget https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kG.pth
            !wget https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kD.pth
            g_pretrained_path = "/content/Applio/f0Ov2Super40kG.pth"
            d_pretrained_path = "/content/Applio/f0Ov2Super40kD.pth"
        elif sample_rate == 48000:
            raise RuntimeError("O Ov2 não tem sample rate de 48k, escolha outro pretreino")
    elif pretreino_escolhido == "Nanashi v1":
        if sample_rate == 32000:
            !wget https://huggingface.co/shiromiya/nanashi-pretrain/resolve/main/v1/G_nanashi_v1.pth
            !wget https://huggingface.co/shiromiya/nanashi-pretrain/resolve/main/v1/D_nanashi_v1.pth
            g_pretrained_path = "/content/Applio/G_nanashi_v1.pth"
            d_pretrained_path = "/content/Applio/D_nanashi_v1.pth"
        elif sample_rate == 40000:
            raise RuntimeError("O Nanashi v1 não tem sample rate de 40k, escolha outro pretreino")
        elif sample_rate == 48000:
            raise RuntimeError("O Nanashi v1 não tem sample rate de 48k, escolha outro pretreino")
    elif pretreino_escolhido == "Nanashi v1.7":
        if sample_rate == 32000:
            !wget https://huggingface.co/shiromiya/nanashi-pretrain/resolve/main/v1.7/G_nanashi_v1_7.pth
            !wget https://huggingface.co/shiromiya/nanashi-pretrain/resolve/main/v1.7/D_nanashi_v1_7.pth
            g_pretrained_path = "/content/Applio/G_nanashi_v1_7.pth"
            d_pretrained_path = "/content/Applio/D_nanashi_v1_7.pth"
        elif sample_rate == 40000:
            raise RuntimeError("O Nanashi v1.7 não tem sample rate de 40k, escolha outro pretreino")
        elif sample_rate == 48000:
            raise RuntimeError("O Nanashi v1.7 não tem sample rate de 48k, escolha outro pretreino")
    else:
        raise ValueError("Pretreino desconhecido")

if pretreino_customizado:
    baixar_modelos(pretreino_escolhido, sample_rate)

if "pretrained" not in globals():
    pretrained = True

if "pretreino_customizado" not in globals():
    pretreino_customizado = False

if "g_pretrained_path" not in globals():
    g_pretrained_path = "Custom Path"

if "d_pretrained_path" not in globals():
    d_pretrained_path = "Custom Path"


def start_train():
    if tensorboard == True:
        %load_ext tensorboard
        %tensorboard --logdir /content/Applio/logs/
    !python core.py train --model_name "{nome_do_modelo}" --rvc_version "{versao_do_rvc}" --save_every_epoch "{salvar_a_cada_epoch}" --save_only_latest "{salvar_somente_ultimo_G_e_D}" --save_every_weights "{salvar_cada_weights}" --total_epoch "{total_de_epoch}" --sample_rate "{sample_rate}" --batch_size "{batch_size}" --gpu "{gpu}" --pitch_guidance "{orientação_de_pitch}" --pretrained "{pretrained}" --custom_pretrained "{pretreino_customizado}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{detector_de_overtraining}" --overtraining_threshold "{limiar_do_overtraining}" --sync_graph "{sincronizar_graficos}" --cache_data_in_gpu "{dados_em_cache_na_gpu}"


server_thread = threading.Thread(target=start_train)
server_thread.start()

if auto_backups:
    backup_files()
else:
    while True:
        time.sleep(10)

In [ ]:
#@title ### Treinar o index
!python core.py index --model_name "{nome_do_modelo}" --rvc_version "{versao_do_rvc}"

# Continuar Treino

In [ ]:
#@title ## Carregar Backup
from google.colab import drive
import os
import shutil

# @markdown Coloque o mesmo nome que colocou em nome do modelo
nome_do_modelo = ""  # @param {type:"string"}
source_path = "/content/drive/MyDrive/RVC_Backup/" + nome_do_modelo
destination_path = "/content/Applio/logs/" + nome_do_modelo
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/RVC_Backup/", backup_timestamps_file)
    time__ = os.path.join("/content/Applio/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

In [ ]:
#@title ## Setar variaveis
# @markdown ### ➡️ Use as mesmas configurações que usou antes
model_name = ""  # @param {type:"string"}
sample_rate = 32000  # @param [32000, 40000, 48000] {allow-input: false}
rvc_version = "v2"  # @param ["v2", "v1"] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}